# Pipeline Execution Example

**Pipeline Execution Example**

This example demonstrates the three execution modes for SQL pipelines:
1. Synchronous native execution with pipeline.run()
2. Asynchronous native execution with pipeline.async_run()
3. Airflow DAG generation with pipeline.to_airflow_dag()

### Imports

In [1]:
import asyncio
import time
from datetime import datetime

from clgraph.pipeline import Pipeline

### Example 1: Synchronous Native Execution

Demonstrate synchronous pipeline execution using `pipeline.run()`.

In [2]:
# Define a simple pipeline
queries = [
    (
        "staging_orders",
        """
        CREATE TABLE staging.orders AS
        SELECT
            order_id,
            customer_id,
            order_date,
            amount
        FROM raw.orders
        WHERE order_date >= CURRENT_DATE() - 7
    """,
    ),
    (
        "daily_revenue",
        """
        CREATE TABLE analytics.daily_revenue AS
        SELECT
            DATE(order_date) as date,
            COUNT(*) as order_count,
            SUM(amount) as total_revenue
        FROM staging.orders
        GROUP BY DATE(order_date)
    """,
    ),
    (
        "customer_metrics",
        """
        CREATE TABLE analytics.customer_metrics AS
        SELECT
            customer_id,
            COUNT(*) as order_count,
            SUM(amount) as total_spend
        FROM staging.orders
        GROUP BY customer_id
    """,
    ),
]

# Create pipeline
pipeline = Pipeline(queries, dialect="bigquery")


# Mock executor for demo (in real use, this would execute SQL on your database)
def execute_sql(sql: str):
    """Mock executor - simulates SQL execution"""
    if "CREATE TABLE" in sql:
        table_start = sql.find("CREATE TABLE") + 13
        table_end = sql.find("AS", table_start)
        table_name = sql[table_start:table_end].strip()
        print(f"      [Executing] {table_name}")
        time.sleep(0.1)  # Simulate query execution time


# Run pipeline synchronously
print("EXAMPLE 1: Synchronous Native Execution (pipeline.run())")
print("=" * 70)
result = pipeline.run(executor=execute_sql, max_workers=4, verbose=True)

print("\nResult Summary:")
print(f"  Completed: {len(result['completed'])} queries")
print(f"  Failed: {len(result['failed'])} queries")
print(f"  Total time: {result['elapsed_seconds']:.2f}s")

EXAMPLE 1: Synchronous Native Execution (pipeline.run())
🚀 Starting pipeline execution (3 queries)

📊 Level 1: 1 queries
      [Executing] staging.orders
  ✅ staging_orders

📊 Level 2: 2 queries
      [Executing] analytics.daily_revenue
      [Executing] analytics.customer_metrics


  ✅ daily_revenue
  ✅ customer_metrics

✅ Pipeline completed in 0.21s
   Successful: 3
   Failed: 0

Result Summary:
  Completed: 3 queries
  Failed: 0 queries
  Total time: 0.21s


### Example 2: Asynchronous Native Execution

Demonstrate asynchronous pipeline execution using `pipeline.async_run()`.

**Note:** In Jupyter notebooks, we use `await` directly instead of `asyncio.run()`.

In [3]:
# Define queries for async execution
queries_async = [
    (
        "staging_orders",
        "CREATE TABLE staging.orders AS SELECT * FROM raw.orders",
    ),
    (
        "daily_revenue",
        "CREATE TABLE analytics.daily_revenue AS SELECT DATE(order_date) as date, SUM(amount) FROM staging.orders GROUP BY 1",
    ),
    (
        "customer_metrics",
        "CREATE TABLE analytics.customer_metrics AS SELECT customer_id, COUNT(*) FROM staging.orders GROUP BY 1",
    ),
]

pipeline_async = Pipeline(queries_async, dialect="bigquery")


# Async executor
async def async_execute_sql(sql: str):
    """Async mock executor"""
    if "CREATE TABLE" in sql:
        table_start = sql.find("CREATE TABLE") + 13
        table_end = sql.find("AS", table_start)
        table_name = sql[table_start:table_end].strip()
        print(f"      [Async Executing] {table_name}")
        await asyncio.sleep(0.1)  # Simulate async query execution


print("EXAMPLE 2: Asynchronous Native Execution (pipeline.async_run())")
print("=" * 70)

# In Jupyter, we use await directly
result_async = await pipeline_async.async_run(
    executor=async_execute_sql, max_workers=4, verbose=True
)

print("\nAsync Result Summary:")
print(f"  Completed: {len(result_async['completed'])} queries")
print(f"  Failed: {len(result_async['failed'])} queries")
print(f"  Total time: {result_async['elapsed_seconds']:.2f}s")

EXAMPLE 2: Asynchronous Native Execution (pipeline.async_run())
🚀 Starting async pipeline execution (3 queries)

📊 Level 1: 1 queries
      [Async Executing] staging.orders
  ✅ staging_orders

📊 Level 2: 2 queries
      [Async Executing] analytics.daily_revenue
      [Async Executing] analytics.customer_metrics


  ✅ daily_revenue
  ✅ customer_metrics

✅ Pipeline completed in 0.20s
   Successful: 3
   Failed: 0

Async Result Summary:
  Completed: 3 queries
  Failed: 0 queries
  Total time: 0.20s


### Example 3: Airflow DAG Generation

Demonstrate Airflow DAG generation using `pipeline.to_airflow_dag()`.

**Note:** This requires Apache Airflow to be installed.

In [4]:
# Define queries for Airflow DAG
queries_airflow = [
    (
        "staging_orders",
        "CREATE TABLE staging.orders AS SELECT * FROM raw.orders",
    ),
    (
        "daily_revenue",
        "CREATE TABLE analytics.daily_revenue AS SELECT DATE(order_date) as date, SUM(amount) FROM staging.orders GROUP BY 1",
    ),
]

pipeline_airflow = Pipeline(queries_airflow, dialect="bigquery")


# Executor function (would execute SQL in production)
def airflow_execute_sql(sql: str):
    """Executor for Airflow tasks"""
    print(f"Would execute: {sql[:50]}...")


print("EXAMPLE 3: Airflow DAG Generation (pipeline.to_airflow_dag())")
print("=" * 70)

# Generate Airflow DAG with advanced parameters
try:
    dag = pipeline_airflow.to_airflow_dag(
        executor=airflow_execute_sql,
        dag_id="revenue_pipeline",
        schedule="@daily",
        start_date=datetime(2024, 1, 1),
        description="Daily revenue analytics pipeline",
        catchup=False,
        max_active_runs=3,
        max_active_tasks=10,
        tags=["analytics", "revenue", "daily"],
        default_view="graph",
        orientation="LR",
    )

    print("\n✅ Airflow DAG generated successfully!")
    print(f"   DAG ID: {dag.dag_id}")
    print(f"   Description: {dag.description}")
    print("   Schedule: @daily")
    print(f"   Max Active Runs: {dag.max_active_runs}")
    print(f"   Tags: {dag.tags}")
    print(f"   Tasks: {len(dag.task_dict)} tasks")
    print(f"   Task IDs: {list(dag.task_dict.keys())}")
    print("\nTo use this DAG:")
    print("1. Save this to your Airflow dags/ folder")
    print("2. Airflow will automatically detect and schedule it")
    print("3. View in Airflow UI: http://localhost:8080/dags/revenue_pipeline")

except ImportError:
    print("\n⚠️  Airflow not installed")
    print("   Install with: pip install apache-airflow")
    print("   This is optional - you can still use run() and async_run()")

EXAMPLE 3: Airflow DAG Generation (pipeline.to_airflow_dag())

⚠️  Airflow not installed
   Install with: pip install apache-airflow
   This is optional - you can still use run() and async_run()


### Summary

This example demonstrated three ways to execute SQL pipelines:

1. **Synchronous** (`pipeline.run()`): Simple, blocking execution
2. **Asynchronous** (`pipeline.async_run()`): Non-blocking, concurrent execution
3. **Airflow DAG** (`pipeline.to_airflow_dag()`): Generate production-ready DAGs

All three methods use the same pipeline definition - write once, run anywhere!

### Visualize Pipeline Lineage

Display the simplified column lineage for the execution pipelines.

In [ ]:
import shutil

from clgraph import visualize_pipeline_lineage

if shutil.which("dot") is None:
    print("⚠️  Graphviz not installed. Install with: brew install graphviz")
else:
    print("Sync Pipeline - Simplified Lineage:")
    display(visualize_pipeline_lineage(pipeline.column_graph.to_simplified()))

    print("\nAsync Pipeline - Simplified Lineage:")
    display(visualize_pipeline_lineage(pipeline_async.column_graph.to_simplified()))

    print("\nAirflow Pipeline - Simplified Lineage:")
    display(visualize_pipeline_lineage(pipeline_airflow.column_graph.to_simplified()))